In [2]:
import pandas as pd
import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import operator
import numpy as np

/Users/yishh/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
G = nx.read_edgelist("../0_cleaning/max_wcc_graph.edgelist", create_using=nx.DiGraph(), delimiter="\t", data=True)

In [ ]:
# post_2005_edges = pd.read_csv("../0_cleaning/max_wcc_graph.edgelist", low_memory=False)

In [40]:
dates = nx.get_edge_attributes(G, 'date')
# weights = nx.get_edge_attributes(G, 'weight')

flat_table = map(lambda x: [x[0], x[1], dates[x]] , dates)
wcc_df = pd.DataFrame(flat_table)
wcc_df.columns = ["investor", "company", "invest_date"]
wcc_df["invest_date"].apply(pd.to_datetime)
post_2005_edges = wcc_df
post_2005_edges["invest_date"] = post_2005_edges["invest_date"].apply(pd.to_datetime)
post_2005_edges = post_2005_edges.sort_values(['invest_date'])

In [42]:
total_periods = 132
rng = pd.date_range('1/1/2005', periods= total_periods, freq='M')
months_per_period = 3
print rng
for i in range(1,total_periods/months_per_period):
    period = i * months_per_period
    curr_rng = rng[:period]
    curr_start = curr_rng[0]
    curr_end = curr_rng[-1]
    mask = (post_2005_edges['invest_date'] >= curr_start) & (post_2005_edges['invest_date'] <= curr_end)
    period_snapshot = post_2005_edges[mask]
    rtn_string = curr_end.strftime('%Y%m%d')
    file_name = "../../df_wccsnapshots/cumulative/enddate" + rtn_string[:10] + ".csv"
#     file_name = file_name.encode('utf-8').strip().replace("-","")
    period_snapshot.to_csv(file_name, index='false', encoding='utf-8')

DatetimeIndex(['2005-01-31', '2005-02-28', '2005-03-31', '2005-04-30',
               '2005-05-31', '2005-06-30', '2005-07-31', '2005-08-31',
               '2005-09-30', '2005-10-31',
               ...
               '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30',
               '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31',
               '2015-11-30', '2015-12-31'],
              dtype='datetime64[ns]', length=132, freq='M')


In [44]:
total_periods = 132
rng = pd.date_range('1/1/2005', periods= total_periods, freq='M')
months_per_period = 3
print rng
for i in range(1,total_periods/months_per_period):
    period = i * months_per_period
    curr_rng = rng[period - months_per_period:period]
    curr_start = curr_rng[0]
    curr_end = curr_rng[-1]
    mask = (post_2005_edges['invest_date'] >= curr_start) & (post_2005_edges['invest_date'] <= curr_end)
    period_snapshot = post_2005_edges[mask]
    rtn_string = curr_end.strftime('%Y%m%d')
    file_name = "../../df_wccsnapshots/diff/enddate" + rtn_string[:10] + ".csv"
#     file_name = file_name.encode('utf-8').strip().replace("-","")
    period_snapshot.to_csv(file_name, index='false', encoding='utf-8')

DatetimeIndex(['2005-01-31', '2005-02-28', '2005-03-31', '2005-04-30',
               '2005-05-31', '2005-06-30', '2005-07-31', '2005-08-31',
               '2005-09-30', '2005-10-31',
               ...
               '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30',
               '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31',
               '2015-11-30', '2015-12-31'],
              dtype='datetime64[ns]', length=132, freq='M')


In [23]:
#create graph edge lists from the csv's generated above.

def customGraphGenerator(filename, attribute_key):
    g = nx.DiGraph()
    
    with open(filename, 'r') as f:
        for line in f.readlines():
            
            # Parse Line
            edge = line.strip().split(",")
            
            # Create Attributes
            attributes = dict(zip(attribute_key, edge[2:]))
            
            # Add Edge
            g.add_edge(edge[0], edge[1], attr_dict=attributes)

    return g

total_periods = 132
rng = pd.date_range('1/1/2005', periods= total_periods, freq='M')
months_per_period = 3
print rng
for i in range(1,total_periods/months_per_period):
    period = i * months_per_period
    curr_rng = rng[:period]
    curr_start = curr_rng[0]
    curr_end = curr_rng[-1]
    rtn_string = curr_end.strftime('%Y%m%d')
    input_file_name = "../../df_wccsnapshots/cumulative/enddate" + rtn_string[:10] + ".csv"
    output_file_name = "../../graph_edgelist_wccsnapshots/cumulative/enddate" + rtn_string[:10] + ".csv"

    attributes = ["weight", "date"] # All additiona attributes, order matters
    
    
    g = customGraphGenerator(input_file_name, attribute_key = attributes)

    # Write the edge to a NetworkX format; delimiter='\t'
    nx.write_edgelist(g, output_file_name, delimiter="\t", data=True)



DatetimeIndex(['2005-01-31', '2005-02-28', '2005-03-31', '2005-04-30',
               '2005-05-31', '2005-06-30', '2005-07-31', '2005-08-31',
               '2005-09-30', '2005-10-31',
               ...
               '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30',
               '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31',
               '2015-11-30', '2015-12-31'],
              dtype='datetime64[ns]', length=132, freq='M')


In [25]:
#create graph edge lists from the csv's generated above for the differences

def customGraphGenerator(filename, attribute_key):
    g = nx.DiGraph()
    
    with open(filename, 'r') as f:
        for line in f.readlines():
            
            # Parse Line
            edge = line.strip().split(",")
            
            # Create Attributes
            attributes = dict(zip(attribute_key, edge[2:]))
            
            # Add Edge
            g.add_edge(edge[0], edge[1], attr_dict=attributes)

    return g

total_periods = 132
rng = pd.date_range('1/1/2005', periods= total_periods, freq='M')
months_per_period = 3
print rng
for i in range(1,total_periods/months_per_period):
    period = i * months_per_period
    curr_rng = rng[:period]
    curr_start = curr_rng[0]
    curr_end = curr_rng[-1]
    rtn_string = curr_end.strftime('%Y%m%d')
    input_file_name = "../../df_wccsnapshots/diff/enddate" + rtn_string[:10] + ".csv"
    output_file_name = "../../graph_edgelist_wccsnapshots/diff/enddate" + rtn_string[:10] + ".csv"

    attributes = ["weight", "date"] # All additiona attributes, order matters
    
    
    g = customGraphGenerator(input_file_name, attribute_key = attributes)

    # Write the edge to a NetworkX format; delimiter='\t'
    nx.write_edgelist(g, output_file_name, delimiter="\t", data=True)



DatetimeIndex(['2005-01-31', '2005-02-28', '2005-03-31', '2005-04-30',
               '2005-05-31', '2005-06-30', '2005-07-31', '2005-08-31',
               '2005-09-30', '2005-10-31',
               ...
               '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30',
               '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31',
               '2015-11-30', '2015-12-31'],
              dtype='datetime64[ns]', length=132, freq='M')


In [45]:
g1 = nx.read_edgelist("../../graph_edgelist_wccsnapshots/cumulative/enddate20100630.csv", delimiter="\t", data=True)

In [50]:
g1.nodes(data=True)

[(u'89370', {}),
 (u'89371', {}),
 (u'11547', {}),
 (u'89376', {}),
 (u'89377', {}),
 (u'11549', {}),
 (u'/organization/forticom', {}),
 (u'83893', {}),
 (u'83892', {}),
 (u'19395', {}),
 (u'5981', {}),
 (u'19393', {}),
 (u'5984', {}),
 (u'/organization/svb-india-capital-partners', {}),
 (u'82448', {}),
 (u'103540', {}),
 (u'82441', {}),
 (u'103542', {}),
 (u'103545', {}),
 (u'82446', {}),
 (u'82444', {}),
 (u'/organization/access2net', {}),
 (u'82991', {}),
 (u'88396', {}),
 (u'88390', {}),
 (u'88393', {}),
 (u'46138', {}),
 (u'46135', {}),
 (u'46132', {}),
 (u'/organization/d-e-shaw-co', {}),
 (u'97153', {}),
 (u'97152', {}),
 (u'97150', {}),
 (u'97155', {}),
 (u'89375', {}),
 (u'/person/vadim-ugov', {}),
 (u'36703', {}),
 (u'270', {}),
 (u'79235', {}),
 (u'19971', {}),
 (u'19977', {}),
 (u'19975', {}),
 (u'277', {}),
 (u'19979', {}),
 (u'19978', {}),
 (u'16703', {}),
 (u'88020', {}),
 (u'88025', {}),
 (u'88024', {}),
 (u'12015', {}),
 (u'88029', {}),
 (u'12010', {}),
 (u'17258', {})

In [49]:
for node in g1.nodes_iter():
    print node
    print node
    break

89370
89370


In [34]:
#calculate all shortest path lengths
# G = nx.path_graph(5)
all_path_lengths = nx.all_pairs_shortest_path_length(g1)
all_path_lengths_sorted = []

# all_path_lengths
for key in all_path_lengths:
    p = all_path_lengths[key]
    curr_distances =  sorted(list(zip(p.keys(),p.values())), key = lambda x: x[1], reverse = True)
    all_path_lengths_sorted.append(curr_distances)
    
all_path_lengths_sorted

[[(u'23326', 2),
  (u'26906', 2),
  (u'16723', 2),
  (u'59188', 2),
  (u'48722', 2),
  (u'80265', 2),
  (u'4215', 2),
  (u'101515', 2),
  (u'61225', 2),
  (u'2286', 2),
  (u'50373', 2),
  (u'62192', 2),
  (u'11572', 2),
  (u'101592', 2),
  (u'80587', 2),
  (u'102503', 2),
  (u'104589', 2),
  (u'46032', 2),
  (u'14723', 2),
  (u'94281', 2),
  (u'103033', 2),
  (u'85069', 2),
  (u'29800', 2),
  (u'42590', 2),
  (u'33045', 2),
  (u'/organization/mission-ventures', 1),
  (u'89370', 0)],
 [(u'/person/mark-leslie', 1), (u'89371', 0)],
 [(u'/organization/balyasny-asset-management', 1), (u'11547', 0)],
 [(u'70950', 2),
  (u'51633', 2),
  (u'31674', 2),
  (u'44476', 2),
  (u'102088', 2),
  (u'34625', 2),
  (u'55765', 2),
  (u'494', 2),
  (u'81717', 2),
  (u'/organization/diamondhead-ventures', 1),
  (u'89376', 0)],
 [(u'77150', 2),
  (u'93939', 2),
  (u'90625', 2),
  (u'/organization/nomura-phase4-ventures', 1),
  (u'89377', 0)],
 [(u'/person/joel-spiegel', 1), (u'11549', 0)],
 [(u'86876', 1), 

In [ ]:
#calculate number of paths between every other node
# G = nx.path_graph(5)
g_size = len(G)

all_number_paths = np.zeros((g_size,g_size))

for i in range(g_size):
    for j in range(i,g_size):
        if i == j:
            continue
        all_paths = nx.all_simple_paths(G, source=i, target=j)
        num = len(list(all_paths))
        all_number_paths[i,j] = num
        all_number_paths[j,i] = num

all_number_paths


In [ ]:
#calculate number of paths between every other node
# G = nx.path_graph(5)
g_size = len(G)

all_number_paths = np.zeros((g_size,g_size))

adamic_index = nx.adamic_adar_index(G)
adamic_index_list = [p for p in adamic_index]
adamic_index_list


In [ ]:
#once i have generated every graph
#for each snapshot
    #generate a graph
    #for each graph
        #get all shortest paths lengths
        #get number paths between every node
        #get adamic index
        #store it properly
        
#random link predictor
